In [1]:
import os
import numpy as np
import pandas as pd
import ast
import re
import csv

import nltk
#from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
ps = PorterStemmer()
lemma = WordNetLemmatizer()

from sklearn.feature_extraction.text import CountVectorizer


import gensim
from gensim import corpora, models
os.chdir('/home/ekhongl/Codes/DL - Topic Modelling')

In [2]:
dir_src = os.path.join(os.getcwd(), 'data/raw_20news/20news-18828')

dir_src_classes = list( map(lambda x: os.path.join(dir_src, x ), os.listdir(dir_src)) )

In [3]:
dat = []
dat_y = []
dat_y_cat = []

for i in range(0,len(dir_src_classes)):
    
    print('Currently loading the following topic (iteration ' + str(i) + '):\n \t' + dir_src_classes[i])
    dir_src_classes_file = list( map(lambda x: os.path.join(dir_src_classes[i], x), os.listdir(dir_src_classes[i])) )
    
    for ii in range(0, len(dir_src_classes_file)):
        
        dat_y.append(i)
        
        with open(dir_src_classes_file[ii], encoding='ISO-8859-1') as file:
            dat.append(file.read().replace('\n', ' '))

Currently loading the following topic (iteration 0):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/alt.atheism
Currently loading the following topic (iteration 1):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/comp.graphics
Currently loading the following topic (iteration 2):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/comp.os.ms-windows.misc
Currently loading the following topic (iteration 3):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/comp.sys.ibm.pc.hardware
Currently loading the following topic (iteration 4):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/comp.sys.mac.hardware
Currently loading the following topic (iteration 5):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/comp.windows.x
Currently loading the following topic (iteration 6):
 	/home/ekhongl/Codes/DL - Topic Modelling/data/raw_20news/20news-18828/misc.forsale


In [23]:
# mapping sub topics 0-19 to broader topics 0-5
y_map = [3,0,0,0,0,0,5,1,1,1,1,2,2,2,2,3,4,4,4,3]
dat_y2 = [y_map[idx] for idx in dat_y]

In [25]:
#export data
pd.DataFrame( { '_label_granular' : dat_y, 
                '_label_overview' : dat_y2,
                'document' : [' '.join(re.sub('[^a-zA-Z]+', ' ', doc).strip().split()) for doc in dat]}). \
                to_csv('data/raw_20news/20news.csv',
                    index=False, sep=',', encoding='ISO-8859-1')

In [4]:
print('------- Data cleaning -------')                
stopwords_en = stopwords.words('english')
dat_clean = []
for i in range(len(dat)):

    ''' tokenization and punctuation removal '''
    # uses nltk tokenization - e.g. shouldn't = [should, n't] instead of [shouldn, 't]
    tmp_doc = nltk.tokenize.word_tokenize(dat[i].lower())
    
    # split words sperated by fullstops
    tmp_doc_split = [w.split('.') for w in tmp_doc if len(w.split('.')) > 1]
    # flatten list
    tmp_doc_split = [i_sublist for i_list in tmp_doc_split for i_sublist in i_list]
    # clean split words
    tmp_doc_split = [w for w in tmp_doc_split if re.search('^[a-z]+$',w)]
    
    # drop punctuations
    tmp_doc_clean = [w for w in tmp_doc if re.search('^[a-z]+$',w)]
    tmp_doc_clean.extend(tmp_doc_split)

    ''' stop word removal'''
    tmp_doc_clean_stop = [w for w in tmp_doc_clean if w not in stopwords_en]
    #retain only words with 2 characters or more
    tmp_doc_clean_stop = [w for w in  tmp_doc_clean_stop if len(w) >2]
    
    ''' stemming (using the Porter's algorithm)'''
    tmp_doc_clean_stop_stemmed = [ps.stem(w) for w in tmp_doc_clean_stop]
    dat_clean.append(tmp_doc_clean_stop_stemmed)
    
    #print progress
    if i % 100 == 0: print( 'Current progress: ' + str(i) + '/' + str(len(dat)) )


------- Data cleaning -------
Current progress: 0/18828
Current progress: 100/18828
Current progress: 200/18828
Current progress: 300/18828
Current progress: 400/18828
Current progress: 500/18828
Current progress: 600/18828
Current progress: 700/18828
Current progress: 800/18828
Current progress: 900/18828
Current progress: 1000/18828
Current progress: 1100/18828
Current progress: 1200/18828
Current progress: 1300/18828
Current progress: 1400/18828
Current progress: 1500/18828
Current progress: 1600/18828
Current progress: 1700/18828
Current progress: 1800/18828
Current progress: 1900/18828
Current progress: 2000/18828
Current progress: 2100/18828
Current progress: 2200/18828
Current progress: 2300/18828
Current progress: 2400/18828
Current progress: 2500/18828
Current progress: 2600/18828
Current progress: 2700/18828
Current progress: 2800/18828
Current progress: 2900/18828
Current progress: 3000/18828
Current progress: 3100/18828
Current progress: 3200/18828
Current progress: 3300/18

In [28]:
dat = pd.read_csv('data/clean_20news.csv', sep=",")


docs = [ast.literal_eval(doc) for doc in dat['document'].tolist()]

all_words = [word for doc in docs for word in doc]
pd_all_words = pd.DataFrame({'words' : all_words})
pd_unq_word_counts = pd.DataFrame({'count' : pd_all_words.groupby('words').size()}).reset_index().sort('count', ascending = False)

# follow's research paper's top 2000 vocabulary (previously only took data with counts of words more than 150)
pd_unq_word_counts_filtered = pd_unq_word_counts.head(2000)
list_unq_word_filtered = list( pd_unq_word_counts_filtered.ix[:,0] )
len(list_unq_word_filtered)

/home/ekhongl/.conda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


2000

In [33]:
vec = CountVectorizer(input = 'content', lowercase = False, vocabulary = list_unq_word_filtered)

iters = list(range(0,len(docs),500))
iters.append(len(docs))
dtm = np.array([] ).reshape(0,len(list_unq_word_filtered))
for i in range(len(iters)-1):
    dtm = np.concatenate( (dtm, list(map(lambda x: vec.fit_transform(x).toarray().sum(axis=0), docs[iters[i]:iters[i+1]] )) ), axis = 0)
    print( 'Percentage completion: ' + str( (i+1) / (len(iters)-1) ) )

colnames = list_unq_word_filtered
colnames.insert(0,'_label_')

Percentage completion: 0.02631578947368421
Percentage completion: 0.05263157894736842
Percentage completion: 0.07894736842105263
Percentage completion: 0.10526315789473684
Percentage completion: 0.13157894736842105
Percentage completion: 0.15789473684210525
Percentage completion: 0.18421052631578946
Percentage completion: 0.21052631578947367
Percentage completion: 0.23684210526315788
Percentage completion: 0.2631578947368421
Percentage completion: 0.2894736842105263
Percentage completion: 0.3157894736842105
Percentage completion: 0.34210526315789475
Percentage completion: 0.3684210526315789
Percentage completion: 0.39473684210526316
Percentage completion: 0.42105263157894735
Percentage completion: 0.4473684210526316
Percentage completion: 0.47368421052631576
Percentage completion: 0.5
Percentage completion: 0.5263157894736842
Percentage completion: 0.5526315789473685
Percentage completion: 0.5789473684210527
Percentage completion: 0.6052631578947368
Percentage completion: 0.63157894736

In [ ]:
pd.DataFrame(data = np.c_[dat['label'].values, dtm], 
             columns = colnames). \
             to_csv( 'data/dtm_2000_20news.csv', index = False)

In [34]:
pd.DataFrame(data = np.c_[dat_y2, dtm], 
             columns = colnames). \
             to_csv( 'data/dtm_2000_20news_6class.csv', index = False)

In [30]:
list_unq_word_filtered

['edu',
 'subject',
 'com',
 'would',
 'one',
 'use',
 'write',
 'articl',
 'like',
 'get',
 'peopl',
 'know',
 'think',
 'time',
 'say',
 'also',
 'make',
 'work',
 'could',
 'want',
 'good',
 'new',
 'system',
 'year',
 'right',
 'see',
 'need',
 'way',
 'even',
 'may',
 'well',
 'look',
 'thing',
 'problem',
 'god',
 'file',
 'tri',
 'much',
 'mani',
 'first',
 'max',
 'two',
 'question',
 'window',
 'take',
 'call',
 'believ',
 'post',
 'come',
 'anyon',
 'point',
 'program',
 'run',
 'said',
 'seem',
 'mean',
 'help',
 'state',
 'read',
 'pleas',
 'differ',
 'drive',
 'number',
 'thank',
 'someth',
 'find',
 'back',
 'realli',
 'game',
 'sinc',
 'includ',
 'day',
 'still',
 'inform',
 'give',
 'reason',
 'person',
 'univers',
 'christian',
 'gener',
 'go',
 'govern',
 'start',
 'part',
 'last',
 'support',
 'might',
 'sure',
 'ask',
 'let',
 'case',
 'follow',
 'law',
 'never',
 'set',
 'comput',
 'better',
 'imag',
 'interest',
 'must',
 'car',
 'power',
 'key',
 'group',
 'fact'

## reading the exported raw text and labels

In [76]:
df = pd.read_csv('data/raw_20news/20news.csv', sep=",")
df

,_label,document
0,0,From mathew mathew mantis co uk Subject Alt At...
1,0,From mathew mathew mantis co uk Subject Alt At...
2,0,From I dbstu rz tu bs de Benedikt Rosenau Subj...
3,0,From mathew mathew mantis co uk Subject Re uni...
4,0,From strom Watson Ibm Com Rob Strom Subject Re...
5,0,From I dbstu rz tu bs de Benedikt Rosenau Subj...
6,0,From keith cco caltech edu Keith Allan Schneid...
7,0,From I dbstu rz tu bs de Benedikt Rosenau Subj...
8,0,From keith cco caltech edu Keith Allan Schneid...
9,0,From keith cco caltech edu Keith Allan Schneid...
